In [47]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
# 구글드라이브 연결
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [48]:
# 데이터파일 불러오기
df = pd.read_csv('/gdrive/My Drive/3-2_machine_learning/image-10k.csv', header=None)

X = df.iloc[:, 1:].values     # 데이터샘플
y = df.iloc[:, 0].values      # 타겟레이블


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1, stratify=y)

images = ['T-shirt', 'Trouser', 'Sweater', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Boot']

In [49]:
import cv2

hog = cv2.HOGDescriptor((28, 28), (14, 14), (7, 7), (7, 7), 9)

hog_x_train = np.zeros((5000,324),dtype="float64")
hog_x_test = np.zeros_like(hog_x_train)

for i in range(5000):
  hog_x_train[i]=hog.compute(x_train[i].reshape(28,28).astype(np.uint8)).squeeze()

for i in range(5000):
  hog_x_test[i]=hog.compute(x_test[i].reshape(28,28).astype(np.uint8)).squeeze()

x_train = hog_x_train
x_test = hog_x_test

In [50]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)



In [51]:
TOP=0
ELSE=1

coarse_classify=np.ones_like(y_train)
test_coarse=np.ones_like(y_test)

coarse_classify[y_train==0]=TOP
coarse_classify[y_train==2]=TOP
coarse_classify[y_train==3]=TOP
coarse_classify[y_train==4]=TOP
coarse_classify[y_train==6]=TOP


test_coarse[y_test==0]=TOP
test_coarse[y_test==2]=TOP
test_coarse[y_test==3]=TOP
test_coarse[y_test==4]=TOP
test_coarse[y_test==6]=TOP

In [52]:
x_train = x_train[coarse_classify == ELSE]
y_train = y_train[coarse_classify == ELSE]
x_test = x_test[test_coarse == ELSE]
y_test = y_test[test_coarse == ELSE]

In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

'''
knn = KNeighborsClassifier(weights='distance',algorithm='kd_tree',p=1,n_neighbors=3)#0.986
knn = KNeighborsClassifier()

params = {
    'n_neighbors': [3, 5,7,11,13,17],
    'weights': ['uniform', 'distance'],
    'algorithm':['auto','ball_tree','kd_tree','brute'],
    'p':[1,2]
}


grid_tree = GridSearchCV(knn, param_grid=params, cv=3, refit=True)
grid_tree.fit(x_train, y_train)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
knn = grid_tree.best_estimator_
pred_y_knn = knn.predict(x_test)

#knn.fit(x_train, corse_classify)
#pred_y_knn = knn.predict(x_test)
print('knn 분류기 정확도', accuracy_score(y_test, pred_y_knn))#Voting 분류기 정확도 0.99'''
'''
best parameters :  {'algorithm': 'auto', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
best score :  0.9228036538356349
knn 분류기 정확도 0.9236
'''
knn = KNeighborsClassifier(weights='distance',algorithm='kd_tree',p=1,n_neighbors=3)
knn.fit(x_train, y_train)
pred_y_knn = knn.predict(x_test)
print('knn 분류기 정확도', accuracy_score(y_test, pred_y_knn))


knn = KNeighborsClassifier(weights='distance',algorithm='auto',p=1,n_neighbors=3)
knn.fit(x_train, y_train)
pred_y_knn = knn.predict(x_test)
print('knn 분류기 정확도', accuracy_score(y_test, pred_y_knn))

knn 분류기 정확도 0.946
knn 분류기 정확도 0.946


In [54]:
'''
mlp = MLPClassifier()

params = {
    'hidden_layer_sizes': [(100,),(100,10),(512,128,32),(512,128),(20,),(20,2)],
    'activation': ['relu'],
    'batch_size':[500,100,50],
    'solver':['adam'],
    'learning_rate':['adaptive','invscaling'],
    'max_iter':[500,250],
    'alpha':[1e-2,1e-1,1e0,1e1,1e2]
}


grid_tree = GridSearchCV(mlp, param_grid=params, cv=3, refit=True)
grid_tree.fit(x_train, y_train)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
mlp = grid_tree.best_estimator_
pred_y_mlp = mlp.predict(x_test)

#mlp.fit(x_train, corse_classify)
#pred_y_mlp = mlp.predict(x_test)
print('mlp 분류기 정확도', accuracy_score(y_test, pred_y_mlp))#Voting 분류기 정확도 0.99'''
'''best parameters :  {'activation': 'relu', 'alpha': 0.01, 'batch_size': 500, 'hidden_layer_sizes': (512, 128), 'learning_rate': 'invscaling', 'max_iter': 250, 'solver': 'adam'}
best score :  0.9503993443900342
mlp 분류기 정확도 0.952'''

from sklearn.neural_network import MLPClassifier
print(x_train.shape)
mlp = MLPClassifier(hidden_layer_sizes=(512,128,32), activation='relu', batch_size=2500,
                    solver='adam', learning_rate="adaptive",  max_iter=500,alpha=1e-2,random_state=1)#0.964

mlp.fit(x_train, y_train)
pred_y_mlp = mlp.predict(x_test)
print('mlp 분류기 정확도', accuracy_score(y_test, pred_y_mlp))

'''mlp = MLPClassifier(hidden_layer_sizes=(512, 128), activation='relu', batch_size=500,
                    solver='adam', learning_rate="invscaling",  max_iter=250,alpha=1e-2)

mlp.fit(x_train, y_train)
pred_y_mlp = mlp.predict(x_test)
print('mlp 분류기 정확도', accuracy_score(y_test, pred_y_mlp))'''

(2500, 324)
mlp 분류기 정확도 0.964


'mlp = MLPClassifier(hidden_layer_sizes=(512, 128), activation=\'relu\', batch_size=500,\n                    solver=\'adam\', learning_rate="invscaling",  max_iter=250,alpha=1e-2)\n\nmlp.fit(x_train, y_train)\npred_y_mlp = mlp.predict(x_test)\nprint(\'mlp 분류기 정확도\', accuracy_score(y_test, pred_y_mlp))'

In [55]:
from sklearn import svm
svm_clf = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=3e0,tol=1e-1)#0.9878
#svm_clf = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=3e0,tol=1e-1)

'''svm_clf = svm.SVC()

params = {
    'kernel': ['poly','rbf'],
    'gamma': ['auto',1e-2,1e-1,1e0,1e1,1e2],
    'C':[1e-2,1e-1,1e0,1e1,1e2],
    'tol':[1e-2,1e-1,1e0,1e1,1e-3],
    'probability':[True],
    'max_iter':[100,500,-1]
}

grid_tree = GridSearchCV(svm_clf, param_grid=params, cv=3, refit=True)
grid_tree.fit(x_train, y_train)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
svm_clf = grid_tree.best_estimator_'''



svm_clf = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=1e2,tol=1e-2)
svm_clf.fit(x_train, y_train)
pred_y_svm = svm_clf.predict(x_test)
print('svm_clf 분류기 정확도', accuracy_score(y_test, pred_y_svm))#Voting 분류기 정확도 0.99

'''svm_clf = svm.SVC(kernel='poly',gamma='auto',probability=True,C=1e1,tol=1e-1,random_state=3)#0.964
svm_clf.fit(x_train, y_train)
pred_y_svm = svm_clf.predict(x_test)
print('svm_clf 분류기 정확도', accuracy_score(y_test, pred_y_svm))#Voting 분류기 정확도 0.99'''
'''
best parameters :  {'C': 100.0, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 500, 'probability': True, 'tol': 0.01}
best score :  0.9487967849010107
svm_clf 분류기 정확도 0.9476
'''

svm_clf 분류기 정확도 0.9652


"\nbest parameters :  {'C': 100.0, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 500, 'probability': True, 'tol': 0.01}\nbest score :  0.9487967849010107\nsvm_clf 분류기 정확도 0.9476\n"

In [57]:
from sklearn.ensemble import VotingClassifier

for i in range(3):
  mlp = MLPClassifier(hidden_layer_sizes=(512,128,32), activation='relu', batch_size=2500,
                    solver='adam', learning_rate="adaptive",  max_iter=500,alpha=1e-2,random_state=i+1)
  svm_clf1 = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=1e2,tol=1e-2,random_state=i+1)#0.9878
  svm_clf2 = svm.SVC(kernel='poly',gamma='auto',probability=True,C=1e0,tol=1e-1,random_state=i+1)

  vo_clf = VotingClassifier(estimators=[('mlp', mlp),('svm1',svm_clf1),('svm2',svm_clf2)],  voting='soft')

  vo_clf.fit(x_train, y_train)
  pred_y = vo_clf.predict(x_test)
  print('corse 분류기 정확도', accuracy_score(y_test, pred_y))#Voting 분류기 정확도 0.7956

corse 분류기 정확도 0.9676
corse 분류기 정확도 0.9652
corse 분류기 정확도 0.964
